# Demographics
This script is used for parsing demographics data.

Source: https://www.czso.cz/csu/czso/databaze-demografickych-udaju-za-obce-cr

In [1]:
import time
import pandas as pd
import numpy as np
import re
import json
import warnings
import json
import os

warnings.filterwarnings('ignore')

In [3]:
PATH_INTERMEDIATE = '../../data/intermediate/'
PATH_RAW = '../../data/raw/'
PATH_POPULATION = PATH_RAW+'population/'

In [62]:
## NUTS x name of the distict
# https://apl.czso.cz/irso4/export4.jsp?kodcis=101&vazcis=0&vazid=0&expvaz=1768&expatr=43&next=Dal%C5%A1%C3%AD
nuts = pd.read_excel(PATH_RAW + 'CIS101.xlsx')
nuts = nuts[['OKRES_LAUT,C,100', 'CZNUTS,C,15']]
nuts.columns = ['Okres', 'NUTS']
nuts = pd.Series(nuts.Okres.values,index=nuts.NUTS).to_dict()

In [65]:
demo = pd.DataFrame()
for name in os.listdir(PATH_POPULATION):
    district = pd.read_excel(PATH_POPULATION + name)
    district['Okres'] = nuts[name[:-5]]
    demo = demo.append(district)

In [66]:
demo = demo[demo.Rok > 2009]

demo = demo[['Číslo\nobce', 'Název obce', 'Okres', 'Rok', 'Stav 31.12.', 'Narození',
             'Zemřelí', 'Přírůstek přirozený', 'Přistě-\nhovalí',
             'Vystě-\nhovalí', 'Přírůstek migrační', 'Přírůstek celkový']]

demo.columns = ['village_num', 'village_name', 'district', 'year',
                'population', 'born', 'deceased', 'born_deceased',
                'immigrants', 'emigrants', 'migration', 'change_total']

In [68]:
demo = demo[demo.population !='-']
demo = demo.applymap(lambda x: 0 if '-' == str(x) else x)

nums = demo.columns[3:]
demo[nums] = demo[nums].applymap(int)
demo

,village_num,village_name,district,year,population,born,deceased,born_deceased,immigrants,emigrants,migration,change_total
39,539104,Bojanovice,Praha-západ,2010,425,5,8,-3,24,12,12,9
40,539104,Bojanovice,Praha-západ,2011,420,6,12,-6,17,10,7,1
41,539104,Bojanovice,Praha-západ,2012,417,1,4,-3,18,18,0,-3
42,539104,Bojanovice,Praha-západ,2013,432,5,0,5,23,13,10,15
43,539104,Bojanovice,Praha-západ,2014,447,2,6,-4,25,6,19,15
...,...,...,...,...,...,...,...,...,...,...,...,...
3144,585050,Zaječí,Břeclav,2017,1442,18,11,7,27,29,-2,5
3145,585050,Zaječí,Břeclav,2018,1432,19,20,-1,17,26,-9,-10
3146,585050,Zaječí,Břeclav,2019,1424,13,14,-1,34,41,-7,-8
3147,585050,Zaječí,Břeclav,2020,1454,15,13,2,61,33,28,30


In [71]:
demo.to_csv(PATH_INTERMEDIATE+'demographics.csv', index=False)